## Environment Setup

In [20]:
import sys
import os
import datetime
import ipywidgets as widgets
import torch

print('Python %s on %s' % (sys.version, sys.platform))

package_path = os.path.abspath(os.path.join(os.path.expanduser(os.getcwd()), os.pardir))
print(package_path)

"""
Adding the path to the neuralmagic-pytorch extension to the path so it isn't necessary to have it installed
"""
sys.path.extend([package_path])

print('Added current package path to sys.path')
print('Be sure to install from requirements.txt and pytorch separately')


Python 3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0] on linux
/home/dan/oldcode/neuralmagicml-pytorch
Added current package path to sys.path
Be sure to install from requirements.txt and pytorch separately


## Dataset

In [21]:
print('\nEnter the local path where the dataset can be found')

dataset_text = widgets.Text(value='/home/dan/Datasets/imagenette/', placeholder='Enter local path to dataset', description='Dataset Path')
display(dataset_text)

print('\nChoose the batch size to run through the model during train and test runs')
print('(be sure to press enter if/after inputting manually)')
train_batch_size_slider = widgets.IntSlider(
    value=256, min=1, max=1024, step=1, description='Train Batch Size:'
)
display(train_batch_size_slider)
test_batch_size_slider = widgets.IntSlider(
    value=256 if torch.cuda.is_available() else 1, min=1, max=1024, step=1, description='Test Batch Size:'
)
display(test_batch_size_slider)



Enter the local path where the dataset can be found


Text(value='/home/dan/Datasets/imagenette/', description='Dataset Path', placeholder='Enter local path to data…


Choose the batch size to run through the model during train and test runs
(be sure to press enter if/after inputting manually)


IntSlider(value=256, description='Train Batch Size:', max=1024, min=1)

IntSlider(value=256, description='Test Batch Size:', max=1024, min=1)

In [22]:
from neuralmagicML.datasets import ImagenetteDataset, ImageNetDataset, EarlyStopDataset
from torch.utils.data import Dataset, DataLoader

dataset_root = os.path.abspath(os.path.expanduser(dataset_text.value))
print('\nLoading dataset from {}'.format(dataset_root))

if not os.path.exists(dataset_root):
    raise Exception('Folder must exist for dataset at {}'.format(dataset_root))
    
train_batch_size = train_batch_size_slider.value
test_batch_size = test_batch_size_slider.value

print('\nUsing train batch size of {} and test batch size of {}\n'
      .format(train_batch_size, test_batch_size))

train_dataset = ImagenetteDataset(dataset_root, train=True, rand_trans=True)
train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4)
print('train dataset created: \n{}\n'.format(train_dataset))

val_dataset = ImagenetteDataset(dataset_root, train=False, rand_trans=False)
val_data_loader = DataLoader(val_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4)
print('validation test dataset created: \n{}\n'.format(val_dataset))

train_test_dataset = EarlyStopDataset(ImagenetteDataset(dataset_root, train=True, rand_trans=False),
                                      early_stop=len(val_dataset) if len(val_dataset) > 1000 else round(0.1 * len(train_dataset)))
train_test_data_loader = DataLoader(train_test_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4)
print('train test dataset created: \n{}\n'.format(train_test_dataset))



Loading dataset from /home/dan/Datasets/imagenette

Using train batch size of 256 and test batch size of 256

already downloaded imagenette of size ImagenetteSize.s160
train dataset created: 
Dataset ImagenetteDataset
    Number of datapoints: 12894
    Root location: /home/dan/Datasets/imagenette/imagenette-160/train

already downloaded imagenette of size ImagenetteSize.s160
validation test dataset created: 
Dataset ImagenetteDataset
    Number of datapoints: 500
    Root location: /home/dan/Datasets/imagenette/imagenette-160/val

already downloaded imagenette of size ImagenetteSize.s160
train test dataset created: 
Dataset ImagenetteDataset
    Number of datapoints: 1289
    Root location: /home/dan/Datasets/imagenette/imagenette-160/train



## Model Setup

In [23]:
from neuralmagicML.models import resnet18

num_classes = 10 if isinstance(train_dataset, ImagenetteDataset) else 1000
#pretrained = 'imagenette/dense' if isinstance(train_dataset, ImagenetteDataset) else True
pretrained = True
model = resnet18(num_classes=num_classes, pretrained=pretrained)
model_id = '{}-{}'.format(model.__class__.__name__,
                          datetime.datetime.today().strftime('%Y-%m-%d-%H:%M:%S')
                              .replace('-', '.').replace(':', '.'))
print('Created model {}'.format(model.__class__.__name__))

print('\nSet the model id')
model_id_text = widgets.Text(
    value=model_id
)
display(model_id_text)

print('\nChoose the device to run on')
device_choice = widgets.ToggleButtons(
    options=['cuda', 'cpu'] if torch.cuda.is_available() else ['cpu'],
    description='Device'
)
display(device_choice)


Created model ResNet

Set the model id


Text(value='ResNet-2019.08.23.16.49.21')


Choose the device to run on


ToggleButtons(description='Device', options=('cuda', 'cpu'), value='cuda')

## Hyperparams

In [24]:
model_id = model_id_text.value
device = device_choice.value

model = model.to(device)

print('\nchoose which tensors to regularize: the inputs or outputs to each conv layer')
reg_tensor_choice = widgets.ToggleButtons(
    options=['inp', 'out'],
    description='reg tens'
)
display(reg_tensor_choice)

print('\nchoose which regularization function to use: l1, l2, relu for the tensors')
reg_func_choice = widgets.ToggleButtons(
    options=['l1', 'l2', 'relu']
)
display(reg_func_choice)

print('\nchoose the alpha value to use for regularization of the activation values')
alpha_slider = widgets.FloatLogSlider(
    value=0.000018, min=-9, max=-1, step=0.0001, description='alpha'
)
display(alpha_slider)



choose which tensors to regularize: the inputs or outputs to each conv layer


ToggleButtons(description='reg tens', options=('inp', 'out'), value='inp')


choose which regularization function to use: l1, l2, relu for the tensors


ToggleButtons(options=('l1', 'l2', 'relu'), value='l1')


choose the alpha value to use for regularization of the activation values


FloatLogSlider(value=1.8e-05, description='alpha', max=-1.0, min=-9.0, step=0.0001)

In [25]:
from neuralmagicML.utils import lr_analysis, lr_analysis_figure, CrossEntropyLossWrapper
%matplotlib inline
import matplotlib.pyplot as plt

### optimizer definitions
momentum = 0.9
weight_decay = 1e-4
###

# print('\nrunning learning rate analysis...')
# batches_per_sample = round(500 / train_batch_size)  # make sure we have enough sample points per learning rate
# analysis = lr_analysis(model, device, train_data_loader, CrossEntropyLossWrapper(), batches_per_sample,
#                        init_lr=1e-7, final_lr=1e0, sgd_momentum=momentum, sgd_weight_decay=weight_decay)
# lr_analysis_figure(analysis)
# plt.show()

print('\nselect the initial learning rate')
lr_slider = widgets.FloatLogSlider(
    value=0.005, min=-7, max=1, step=0.0001, description='init lr'
)
display(lr_slider)

print('\nselect the number of epochs to train for')
finalize_epochs_text = widgets.IntText(value=90, description='num epochs')
display(finalize_epochs_text)

print('\nselect the final learning rate')
lr_final_slider = widgets.FloatLogSlider(
    value=0.0001, min=-7, max=1, step=0.0001, description='final lr'
)
display(lr_final_slider)

print('\nselect the number of exponential updates to apply to the learning rate over the epochs')
lr_updates_slider = widgets.IntText(value=2, description='lr updates')
display(lr_updates_slider)


select the initial learning rate


FloatLogSlider(value=0.005, description='init lr', max=1.0, min=-7.0, step=0.0001)


select the number of epochs to train for


IntText(value=90, description='num epochs')


select the final learning rate


FloatLogSlider(value=0.0001, description='final lr', max=1.0, min=-7.0, step=0.0001)


select the number of exponential updates to apply to the learning rate over the epochs


IntText(value=2, description='lr updates')

## Setup

In [26]:
from neuralmagicML.sparsity import (
    ASAnalyzerLayer, ASAnalyzerModule, ASRegModifier, LearningRateModifier,
    ScheduledModifierManager, ScheduledOptimizer
)
from neuralmagicML.utils import TopKAccuracy
from tensorboardX import SummaryWriter
from torch.nn.modules.conv import _ConvNd
from torch.nn import Linear
from torch.nn import ReLU
from torch import optim

reg_tens = reg_tensor_choice.value
reg_func = reg_func_choice.value
alpha = alpha_slider.value

print('using AS reg params of reg_tens:{} reg_func:{} and alpha:{}'
     .format(reg_tens, reg_func, alpha))

lr_init = lr_slider.value
lr_final = lr_final_slider.value
epochs = finalize_epochs_text.value
lr_updates = lr_updates_slider.value
lr_update_freq = epochs / (lr_updates + 1.0)
lr_gamma = (lr_final / lr_init) ** (1 / lr_updates)
print('using lr params of init:{} final:{} epochs:{} updates:{} update_freq:{} gamma:{}'
      .format(lr_init, lr_final, epochs, lr_updates, lr_update_freq, lr_gamma))

lr_modifier = LearningRateModifier(lr_class='ExponentialLR', lr_kwargs={'gamma': lr_gamma},
                                   start_epoch=0.0, end_epoch=epochs,
                                   update_frequency=lr_update_freq)
modify_layers = [name for name, mod in model.named_modules() if isinstance(mod, ReLU)]

# remove the first conv if we are working on the input to each conv
if reg_tens == 'inp':
    modify_layers = modify_layers[1:]
# remove the last conv if we are working on the output from each conv
elif reg_tens == 'out':
    modify_layers = modify_layers[:-1]

as_reg_modifier = ASRegModifier(modify_layers, alpha, reg_func, reg_tens, start_epoch=0.0)

modifier_manager = ScheduledModifierManager([lr_modifier, as_reg_modifier])
print('\nCreated ScheduledModifierManager with exponential lr_modifier with gamma {} and AS reg modifier'
      .format(lr_gamma))

optimizer = optim.SGD(model.parameters(), lr_slider.value, momentum=momentum,
                      weight_decay=weight_decay, nesterov=True)
optimizer = ScheduledOptimizer(optimizer, model, modifier_manager, steps_per_epoch=len(train_dataset))
print('\nCreated scheudled optimizer with initial lr: {}, momentum: {}, weight decay: {}'
      .format(lr_slider.value, momentum, weight_decay))

loss = CrossEntropyLossWrapper(extras={'top1acc': TopKAccuracy(1)})
print('\nCreated loss wrapper\n{}'.format(loss))

logs_dir = os.path.abspath(os.path.expanduser(os.path.join('.', 'model_training_logs', model_id)))

if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

writer = SummaryWriter(logdir=logs_dir, comment='imagenette training')
print('\nCreated summary writer logging to \n{}'.format(logs_dir))


using AS reg params of reg_tens:out reg_func:l1 and alpha:1.8e-05
using lr params of init:0.005 final:0.0001 epochs:90 updates:2 update_freq:30.0 gamma:0.1414213562373095

Created ScheduledModifierManager with exponential lr_modifier with gamma 0.1414213562373095 and AS reg modifier

Created scheudled optimizer with initial lr: 0.005, momentum: 0.9, weight decay: 0.0001

Created loss wrapper
CrossEntropyLossWrapper(Loss: cross_entropy; Extras: TopKAccuracy)

Created summary writer logging to 
/home/dan/oldcode/neuralmagicml-pytorch/notebooks/model_training_logs/ResNet-2019.08.23.16.49.21-relu-out


## Training

In [ ]:
from tqdm import tqdm
import math
from neuralmagicML.models import save_model


def test_epoch(model, data_loader, loss, device, epoch):
    model.eval()
    results = {}
    
    with torch.no_grad():
        for batch, (*x_feature, y_lab) in enumerate(tqdm(data_loader)):
            y_lab = y_lab.to(device)
            x_feature = tuple([dat.to(device) for dat in x_feature])
            batch_size = y_lab.shape[0]
            y_pred = model(*x_feature)
            losses = loss(x_feature, y_lab, y_pred)
            
            for key, val in losses.items():
                if key not in results:
                    results[key] = []
                result = val.detach_().cpu()
                result = result.repeat(batch_size)
                results[key].append(result)
                
    return results

def test_epoch_writer(model, data_loader, loss, device, epoch, writer, key):
    losses = test_epoch(model, data_loader, loss, device, epoch)
    
    for loss, values in losses.items():
        val = torch.mean(torch.cat(values))
        writer.add_scalar(key.format(loss), val, epoch)
        print('{}: {}'.format(loss, val))
        
def test_as_values(as_model, data_loader, device, epoch, writer, sample_size=1000):
    as_model.eval()
    as_model.clear_layers()
    as_model.enable_layers()
    sample_count = 0
    
    with torch.no_grad():
        for batch, (*x_feature, y_lab) in enumerate(tqdm(data_loader)):
            y_lab = y_lab.to(device)
            x_feature = tuple([dat.to(device) for dat in x_feature])
            batch_size = y_lab.shape[0]
            y_pred = model(*x_feature)
            sample_count += batch_size
            
            if sample_count >= sample_size:
                break
        
    as_model.disable_layers()
    
    for name, layer in as_model.layers.items():
        writer.add_scalar('Act Sparsity/{}'.format(name), layer.inputs_sparsity_mean, epoch)
    
    as_model.clear_layers()
            

def train_epoch(model, data_loader, optimizer, loss, device, epoch, writer):
    model.train()
    init_batch_size = None
    batches_per_epoch = len(data_loader)
    
    for batch, (*x_feature, y_lab) in enumerate(tqdm(data_loader)):
        y_lab = y_lab.to(device)
        x_feature = tuple([dat.to(device) for dat in x_feature])
        batch_size = y_lab.shape[0]
        if init_batch_size is None:
            init_batch_size = batch_size
        optimizer.zero_grad()
        y_pred = model(*x_feature)
        losses = loss(x_feature, y_lab, y_pred)
        losses['loss'] = optimizer.loss_update(losses['loss']) # update loss with the AS modifier regularization
        losses['loss'].backward()
        optimizer.step(closure=None)
        
        step_count = init_batch_size * (epoch * batches_per_epoch + batch)
        for _loss, _value in losses.items():
            writer.add_scalar('Train/{}'.format(_loss), _value.item(), step_count)
            writer.add_scalar('Train/Learning Rate', optimizer.learning_rate, step_count)
            
print('Training model...')

analyzer_model = ASAnalyzerModule(
    model, [ASAnalyzerLayer(name, division=0, track_inputs_sparsity=True)
            for name, mod in model.named_modules() if isinstance(mod, _ConvNd) or isinstance(mod, Linear)]
)
print('\nCreated AS analyzer module')

print('Running initial validation values for later comparison')
test_epoch_writer(model, val_data_loader, loss, device, -1, writer, 'Test/Validation/{}')
test_as_values(analyzer_model, val_data_loader, device, -1, writer)
            
for epoch in tqdm(range(math.ceil(modifier_manager.max_epochs))):
    print('Starting epoch {}'.format(epoch))
    optimizer.epoch_start()
    train_epoch(model, train_data_loader, optimizer, loss, device, epoch, writer)
    
    print('Completed training for epoch {}, testing validation dataset'.format(epoch))
    test_epoch_writer(model, val_data_loader, loss, device, epoch, writer, 'Test/Validation/{}')
    
    print('Completed testing validation dataset for epoch {}, testing training dataset'.format(epoch))
    test_epoch_writer(model, train_test_data_loader, loss, device, epoch, writer, 'Test/Training/{}')
    
    print('Completed testing validation dataset for epoch {}, testing activation sparsity'.format(epoch))
    test_as_values(analyzer_model, val_data_loader, device, epoch, writer)
        
    optimizer.epoch_end()
    
save_path = os.path.abspath(os.path.expanduser(os.path.join('.', '{}.pth'.format(model_id))))
print('Finished training, saving model to {}'.format(save_path))
save_model(save_path, model, optimizer, epoch)
print('Saved model')










  0%|          | 0/2 [00:00<?, ?it/s]

Training model...

Created AS analyzer module
Running initial validation values for later comparison










 50%|█████     | 1/2 [00:00<00:00,  1.24it/s]







100%|██████████| 2/2 [00:00<00:00,  2.04it/s]







  0%|          | 0/2 [00:00<?, ?it/s]

loss: 2.3126208782196045
top1acc: 12.200007438659668










 50%|█████     | 1/2 [00:00<00:00,  1.40it/s]







100%|██████████| 2/2 [00:00<00:00,  1.86it/s]







  0%|          | 0/90 [00:00<?, ?it/s]








  0%|          | 0/51 [00:00<?, ?it/s]

...

loss: 0.016126228496432304
top1acc: 99.5345230102539
Completed testing validation dataset for epoch 20, testing activation sparsity











 50%|█████     | 1/2 [00:00<00:00,  1.26it/s]








100%|██████████| 2/2 [00:00<00:00,  1.67it/s]







 23%|██▎       | 21/90 [07:08<23:28, 20.41s/it]








  0%|          | 0/51 [00:00<?, ?it/s]

Starting epoch 21











  2%|▏         | 1/51 [00:01<00:50,  1.01s/it]








  4%|▍         | 2/51 [00:01<00:38,  1.26it/s]








  6%|▌         | 3/51 [00:01<00:32,  1.50it/s]








  8%|▊         | 4/51 [00:01<00:25,  1.82it/s]








 10%|▉         | 5/51 [00:02<00:21,  2.12it/s]








 12%|█▏        | 6/51 [00:02<00:19,  2.27it/s]








 14%|█▎        | 7/51 [00:02<00:17,  2.56it/s]








 16%|█▌        | 8/51 [00:03<00:15,  2.76it/s]








 18%|█▊        | 9/51 [00:03<00:15,  2.75it/s]








 20%|█▉        | 10/51 [00:03<00:14,  2.86it/s]








 22%|██▏       | 11/51 [00:04<00:14,  2.81it/s]








 24%|██▎       | 12/51 [00:04<00:12,  3.04it/s]








 25%|██▌       | 13/51 [00:04<00:12,  3.14it/s]








 27%|██▋       | 14/51 [00:05<00:12,  2.98it/s]








 29%|██▉       | 15/51 [00:05<00:11,  3.19it/s]








 31%|███▏      | 16/51 [00:05<00:11,  3.16it/s]








 33%|███▎      | 17/51 [00:06<00:11,  3.00it/s]








 35%|███▌      | 18/51 [00:06<00:10,  3.21it/s]


Completed training for epoch 21, testing validation dataset


In [16]:
test_as_values(analyzer_model, val_data_loader, device, epoch, writer)

100%|██████████| 2/2 [00:01<00:00,  1.11it/s]


In [17]:
print(str(save_path))

/home/dan/oldcode/neuralmagicml-pytorch/notebooks/ResNet-2019.08.23.12.10.30.pth
